## Name: Sanchit Kripalani <br> Batch: M1 <br> Roll No: 31145

In [34]:
# Import necessary libraries
import nltk
from nltk.chat.util import Chat, reflections

In [35]:
# Creating a dictionary of all the items available in the electronics store

item_list = {
    "phone": {
        "iphone": 80000,
        "samsung": 50000,
        "redmi": 14000,
        "oneplus": 30000
    },
    
    "laptop": {
        "macbook": 100000,
        "hp": 70000,
        "dell": 80000,
        "asus": 50000
    }, 
    
    "tv": {
        "samsung": 70000,
        "lg": 120000,
        "micromax": 25000
    }
}

In [36]:
# This is an extended class from Chat for custom behaviour
import random
import re

reflections = {
    "i am": "you are",
    "i was": "you were",
    "i": "you",
    "i'm": "you are",
    "i'd": "you would",
    "i've": "you have",
    "i'll": "you will",
    "my": "your",
    "you are": "I am",
    "you were": "I was",
    "you've": "I have",
    "you'll": "I will",
    "your": "my",
    "yours": "mine",
    "you": "me",
    "me": "you",
}

# The chat will contain logic for processing text data 
# Reflections contains basic input and output pairs
class MyChat:
    def __init__(self, pairs, reflections={}):
        """
        Initialize the chatbot.  Pairs is a list of patterns and responses.  Each
        pattern is a regular expression matching the user's statement or question,
        e.g. r'I like (.*)'.  For each such pattern a list of possible responses
        is given, e.g. ['Why do you like %1', 'Did you ever dislike %1'].  Material
        which is matched by parenthesized sections of the patterns (e.g. .*) is mapped to
        the numbered positions in the responses, e.g. %1.

        :type pairs: list of tuple
        :param pairs: The patterns and responses
        :type reflections: dict
        :param reflections: A mapping between first and second person expressions
        :rtype: None
        """

        self._pairs = [(re.compile(x, re.IGNORECASE), y, z) for (x, y,z) in pairs]
        self._reflections = reflections
        self._regex = self._compile_reflections()


    def _compile_reflections(self):
        sorted_refl = sorted(self._reflections, key=len, reverse=True)
        return re.compile(
            r"\b({})\b".format("|".join(map(re.escape, sorted_refl))), re.IGNORECASE
        )

    def _substitute(self, str):
        """
        Substitute words in the string, according to the specified reflections,
        e.g. "I'm" -> "you are"

        :type str: str
        :param str: The string to be mapped
        :rtype: str
        """

        return self._regex.sub(
            lambda mo: self._reflections[mo.string[mo.start() : mo.end()]], str.lower()
        )

    def _wildcards(self, response, match):
        pos = response.find("%")
        while pos >= 0:
            num = int(response[pos + 1 : pos + 2])
            response = (
                response[:pos]
                + self._substitute(match.group(num))
                + response[pos + 2 :]
            )
            pos = response.find("%")
        return response

    def respond(self, str):
        """
        Generate a response to the user input.

        :type str: str
        :param str: The string to be mapped
        :rtype: str
        """

        # check each pattern
        for (pattern, response, callback) in self._pairs:
            match = pattern.match(str)

            # did the pattern match?
            if match:
                resp = random.choice(response)  # pick a random response
                resp = self._wildcards(resp, match)  # process wildcards

                # fix munged punctuation at the end
                if resp[-2:] == "?.":
                    resp = resp[:-2] + "."
                if resp[-2:] == "??":
                    resp = resp[:-2] + "?"
                if callback:
                    callback(match)
                    
                return resp


    # Hold a conversation with a chatbot
    def converse(self, quit="quit"):
        user_input = ""
        while user_input != quit:
            user_input = quit
            try:
                user_input = input(">")
            except EOFError:
                print(user_input)
            if user_input:
                while user_input[-1] in "!.":
                    user_input = user_input[:-1]
                print(self.respond(user_input))

In [37]:
# Defining a conditional check to find if the item is available
def checkList(match):
    groups = match.groups()
    needed_product = 'NA'
    needed_company = 'NA'

    for item in groups:
        if item in item_list:
            for company in groups:
                if company in item_list[item]:
                    needed_product = item
                    needed_company = company
                    break

    if (needed_company != 'NA'):
        print(f"Yes the product {needed_product} is available in our shop :)")
    else:
        print("I am sorry. The current product is not available at our shop :( ")

In [38]:
def findPrice(match):
    groups = match.groups()
    needed_product = 'NA'
    needed_company = 'NA'

    for item in groups:
        if item in item_list:
            for company in groups:
                if company in item_list[item]:
                    needed_product = item
                    needed_company = company
                    break

    if (needed_company != 'NA'):
        print(f"The price of {needed_product} from {needed_company} is Rs.{item_list[needed_product][needed_company]} :)")
    else:
        print("I am sorry. The current product is not available at our shop :( ")

In [39]:
# Creating regular expressions for questions and their corresponding answers
pairs = [
    [
        r"my name is (.*)",
        ["Hello %1, How are you today?",],
        None
    ],
    [
        r"hi|hey|hello",
        ["Hello!!!", "Hey there!!!",],
        None
    ], 
    [
        r"what is your name?",
        ["My name is Sanbot, but people call me the GOAT of chatbots",],
        None
    ],
    [
        r"how are you ?",
        ["I'm doing well\nHow about You?",],
        None
    ],
    [
        r"sorry (.*)",
        ["No issue sir/ma'am",],
        None
    ],
    [
        r"I am fine",
        ["Great to hear that, How can I help you?",],
        None
    ],
    [
        r"i am doing good",
        ["Nice to hear that","How can I help you?",],
        None
    ],
    [
        r"^.*creator*$",
        ["Sanchit Kripalani is my creator! ","top secret ;)",],
        None
    ],
    [
        r"where do you live?",
        ['I live at 221 Baker\'s Street',],
        None
    ],
    [
        r"is (.*) (.*) (.*) available here?",
        [''],
        checkList
    ],
    [
        r"what is the cost of (.*) (.*)",
        [''],
        findPrice
    ],
    [
        r"^.*shop timing.*$",
        ["The shop timings are: Monday to friday- 9am to 10pm"], 
        None
    ],
    [
        r"^.*name of the shop.*$",
        ["The shop is called San Shop!"], 
        None
    ],
    [
        r"^.*address*$",
        ["The address for the shop is: 221 Baker's street, Westmister, London"], 
        None
    ],
    [
        r"quit",
        ["It was nice talking to you. Hope I could answer your queries properly"],
        None
    ],
]

In [40]:
def chat():
    print("Hi! This is SanBot, A customer Interaction app for San Stores.")
    print("How may I help you today?")
    print("\nPlease note that the format for querying a product is...")
    print("is a/an/the (product_name/product_company) (product_company/product_name) available here?")
    chat = MyChat(pairs, reflections)
    chat.converse()
    
# initiate the conversation
if __name__ == "__main__":
    chat()

Hi! This is SanBot, A customer Interaction app for San Stores.
How may I help you today?

Please note that the format for querying a product is...
is a/an/the (product_name/product_company) (product_company/product_name) available here?
>hi
Hello!!!
>how are you
I'm doing well
How about You?
>i am fine
Great to hear that, How can I help you?
>what is the shop timing
The shop timings are: Monday to friday- 9am to 10pm
>is a samsung phone available here
Yes the product phone is available in our shop :)

>what is the shop address
The address for the shop is: 221 Baker's street, Westmister, London
>what is the cost of samsung phone
The price of phone from samsung is Rs.50000 :)

>quit
It was nice talking to you. Hope I could answer your queries properly
